# Projekt IAU  
## 3. časť - Strojové učenie
### Vypracovali: Martin Kukučka, Martin Jankuliak
### Číslo datasetu: 80
### Podiel práce: 50/50

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
import glob
import re
import datetime
from datetime import datetime
from numpy.random import seed 
from numpy.random import randn 
from numpy import mean
from numpy import std
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from numpy import exp
from sklearn.preprocessing import PowerTransformer
from matplotlib import pyplot

from numpy import isnan
from pandas import read_csv
from sklearn.impute import KNNImputer

import category_encoders as ce

from numpy import percentile

import scipy as sc
from pandas import read_csv


from sklearn.preprocessing import PowerTransformer

Funkcia, ktorá nahradí chýbajúce hodnoty v stĺpci column modusom (najviac vyskytovaný element). Funkcia využíva SimpleImputer.

In [3]:
def get_mode(data_frame, column):
    data = data_frame[column].values.reshape(-1, 1)
    imp = SimpleImputer(missing_values=np.nan, strategy="most_frequent")
    data_frame[column] = imp.fit_transform(data)
    return data_frame

Funkcia, ktorá nahradí chýbajúce hodnoty v stĺpci column mediánom.

In [4]:
def get_median(data_frame, column):
    median = data_frame[column].median()
    data_frame[column].fillna(median, inplace = True)
    return data_frame

Funkcia, ktorá nahradí chýbajúce hodnoty v stĺpci column priemerom.

In [5]:
def get_mean(data_frame, column):
    mean = data_frame[column].mean()
    data_frame[column].fillna(mean, inplace = True)
    return data_frame

Naplnenie chýbajúcich hodnôt v stĺpci / stĺpcoch modusom.

In [6]:
 def fill_mode(data_frame, columns):
    for col in columns:
        data_frame = get_mode(data_frame, col)
    
    return data_frame

Naplnenie chýbajúcich hodnôt v stĺpci / stĺpcoch mediánom.

In [7]:
 def fill_median(data_frame, columns):
    for col in columns:
        data_frame = get_median(data_frame, col)
        
    return data_frame

Naplnenie chýbajúcich hodnôt v stĺpci / stĺpcoch priemerom.

In [8]:
 def fill_mean(data_frame, columns):
    for col in columns:
        data_frame = get_mean(data_frame, col)
        
    return data_frame

Pomocou tejto funkcie prevádzame adresu na numerickú hodnotu tak, že z danej adresy zistíme ZIP code / PSČ a uložíme ho do pôvodného záznamu

In [9]:
def get_zip(data_frame, column):
    for i in range(len(data_frame[column])):
        zipcode = str(data_frame[column][i])
        data_frame.loc[i, column] = zipcode[len(zipcode) - 5:len(zipcode)]
    data_frame[column] = data_frame[column].astype('int')
    return data_frame

Funkcia ráta vek respondentov v záznamoch. Našou analýzou sme zistili, že záznamy sú robené v rokoch 2018 a 2019. Z každého záznamu vezmeme atribút age a ak je hodnota tohto atribútu záporná alebo trojciferná (uvažujeme ako outliery), tak pomocou atribútu date-of-birth vypočítame nový vek. Ak nový vek zostane naďalej záporný alebo trojciferný, tak jednoducho tieto záznamy vymažeme. 

In [10]:
def calculate_age(data_frame, date_of_birth, age):
    for i in range(len(data_frame[date_of_birth])):
        curr_age = int(data_frame[age][i])
        
        if curr_age < 0 or curr_age > 99:
            year = data_frame[date_of_birth][i]
            year = year[:4]            
            new_age = 2019 - int(year)
            
            if new_age < 0 or new_age > 99:
                data_frame = data_frame.drop(i)
            else:
                data_frame.loc[i, age] = str(new_age)
                
    data_frame = data_frame.reset_index()
    data_frame = data_frame.drop('index', 1)        
    return data_frame

Funkcia, ktorá zjednotí duplikáty.

In [11]:
def unify_rows(data_frame):
    aggregation_functions = {}
    for col in data_frame.columns:
        aggregation_functions[col] = "first" 

    return data_frame.groupby(data_frame['name'], as_index=False).aggregate(aggregation_functions)

Pomocou tejto funkcie sme nahradzovali formáty hodnôt iba jednou hodnotou. Napr. hodnoty v atribúte pregnant (F, FALSE, f) sme nahradili hodnotou f.

In [12]:
def cleanse_some_values(data_frame, column, change_from, change_to):
    for i in range(len(change_from)):
        data_frame.loc[data_frame[column] == change_from[i], column] = change_to

V tejto funkcií sme pre každý atribút, kde sme uznali, že by bolo vhodné, zjednotili dáta. Vyuzili sme pipeline-u.

In [13]:
def unify_values(data_frame):
    unify_ppl = Pipeline([
                    ('1. step',  cleanse_some_values(data_frame, 'pregnant', ["F", "FALSE"], "f")),
                    ('2. step', cleanse_some_values(data_frame, 'pregnant', ["T", "TRUE"], "t")),

                    ('3. step', cleanse_some_values(data_frame, 'marital-status', [" Widowed", " Never-married", " Divorced"], "single")),
                    ('4. step',cleanse_some_values(data_frame, 'marital-status', [" Married-civ-spouse", " Separated", " Married-AF-spouse", " Married-spouse-absent"], "married")),

                    ('5. step', cleanse_some_values(data_frame, 'education', [" 1st-4th", " 5th-6th", " 7th-8th", " 9th", " 10th", " 11th", " 12th"], "Primary school")),
                    ('6. step', cleanse_some_values(data_frame, 'education', [" Bachelors", " Masters", " Doctorate"], "College")),
                    ('7. step', cleanse_some_values(data_frame, 'education', [" Assoc-voc", " HS-grad", " Prof-school", " Assoc-acdm", " Some-college"], "Highschool")),
                    ('8. step', cleanse_some_values(data_frame, 'education', [" Preschool"], "Preschool")),

                    ('9. step', cleanse_some_values(data_frame, 'workclass', [" Private"], "private")),
                    ('10. step', cleanse_some_values(data_frame, 'workclass', [" Self-emp-not-inc", " Self-emp-inc", "self-emp-inc", "self-emp-not-inc"], "self-emp")),
                    ('11. step', cleanse_some_values(data_frame, 'workclass', [" Never-worked", " Without-pay"], "not-working")),
                    ('12. step', cleanse_some_values(data_frame, 'workclass', [" Local-gov", "local-gov", " State-gov", "state-gov", " Federal-gov", "federal-gov"], "government")),
                    ('13. step', cleanse_some_values(data_frame, 'workclass', [" ?", "?"], np.nan))
                    ])
    unify_ppl = unify_ppl.fit(data_frame)
    data_frame = unify_ppl.transform(data_frame)
    return data_frame

Funkcia na zjednotenie formátu dátumov vo formáte YEAR-MONTH-DAY

In [14]:
def change_date_format(data_frame, column):
    for i in range(len(data_frame[column])):
        date_format = str(data_frame[column][i])
        date_format = date_format.replace('/', '-')
        if date_format[2] == '-' and len(date_format) < 10:
            temp_year = int(date_format[:2])
            if(temp_year <= 20):
                if(temp_year < 10):
                    year = '200'+str(temp_year)
                else:
                    year = '20'+str(temp_year)
            else:
                year = '19'+str(temp_year)
            data_frame.loc[i, column] = year+date_format[2:10]
        elif date_format[2] == '-' and len(date_format) == 10:
            day = date_format[:2]
            month = date_format[3:5]
            year = date_format[6:10]
            data_frame.loc[i, column] = year+'-'+month+'-'+day
        else:
            data_frame.loc[i, column] = date_format[:10]
    return data_frame

Funkcia zmaže outliery (v našom zadaní sme sa snažili outliery radšej nahrádzať ako mazať, preto nebolo potrebné túto funkciu použiť).

In [15]:
# def remove_outliers(data_frame, column):
#     a = data_frame[column]
#     lower = a.quantile(0.25) - 1.5 * stats.iqr(a)
#     upper = a.quantile(0.75) + 1.5 * stats.iqr(a)
    
#     outliers = a[(a > upper) | (a < lower)]
#     data_frame = data_frame.drop(outliers.index)
    
#     data_frame = data_frame.reset_index()
#     data_frame = data_frame.drop('index', 1)
    
#     return data_frame

Ďalšia funkcia na zmazanie outlierov (Interquartile Range Method).

In [16]:
# def remove_outliers_iqr(data_frame, column):
#     seed(1)
#     data = data_frame[column]
#     sns.distplot(data)

#     # calculate interquartile range
#     q25, q75 = percentile(data, 25), percentile(data, 75)
#     iqr = q75 - q25
#     print('Percentiles: 25th=%.3f, 75th=%.3f, IQR=%.3f' % (q25, q75, iqr)) 

#     # calculate the outlier cutoff
#     cut_off = iqr * 1.5
#     lower, upper = q25 - cut_off, q75 + cut_off
#     print('cutoff=', cut_off, 'lower=', lower, 'upper=', upper)

#     # identify outliers
#     outliers = data[(data > upper) | (data < lower)]
#     print('Identified outliers: %d' % len(outliers))

#     # remove outliers
#     outliers_removed = [x for x in data if x >= lower and x <= upper] 
#     print('Non-outlier observations: %d' % len(outliers_removed))

#     sns.distplot(outliers_removed)
    
#     data_frame = data_frame.drop(outliers.index)
    
#     data_frame = data_frame.reset_index()
#     data_frame = data_frame.drop('index', 1)

#     return data_frame

Funkcia slúži na zmenu hodnoty "t" atribútu pregnant na "f" pre mužov (" Male" z atribútu sex).

In [17]:
def man_not_pregnant(data_frame, sex, pregnant):
    for i in range(len(data_frame[sex])):
        if(data_frame[sex][i] == " Male") and data_frame[pregnant][i] != "f":
            data_frame.loc[i, pregnant] = "f"
    return data_frame

Funkcia transformácie numerických dát, aby boli dáta lepšie distribuované. Zvolili sme metódu yeo-johnson, pretože pracuje nielen s pozitívnymi ale aj s negatívnymi hodnotami.

In [18]:
def transform_data(data_frame, columns):
    for col in columns:
        data = data_frame[col].values.reshape(-1, 1)
        power = PowerTransformer(method='yeo-johnson', standardize=True) 
        data_frame[col] = power.fit_transform(data)
        
    return data_frame

Funkcia mení kategorické dáta na numerické. Typ udáva akou formou prebieha kódovanie.

In [19]:
def encoding(data_frame, column, typ):
    data = data_frame[column]

    if typ == 0:
        ce_ordinal = ce.OneHotEncoder(cols=[column])
        data_frame[column] = ce_ordinal.fit_transform(data)
        
    elif typ == 1:
        ce_ordinal = ce.OrdinalEncoder(cols=[column])
        data_frame[column] = ce_ordinal.fit_transform(data)
    
    return data_frame

Funkcia mení kategorické dáta atribútu pregnant na numerické 0 - not pregnant, 1 - pregnant

In [20]:
def encode_pregnant(data_frame):
    cleanse_some_values(data_frame, 'pregnant', ["f"], 0)
    cleanse_some_values(data_frame, 'pregnant', ["t"], 1)
    data_frame['pregnant'] = data_frame['pregnant'].astype('int')
    return data_frame

Zakódovanie atribútov obsahujúcich kategorické hodnoty. 

In [21]:
def encode_objects(data_frame):
    data_frame = encoding(data_frame, 'sex', 0)
    data_frame = encoding(data_frame, 'race', 1)
    data_frame = encoding(data_frame, 'marital-status', 1)
    data_frame = encoding(data_frame, 'occupation', 1)
    data_frame = encode_pregnant(data_frame)
    data_frame = encoding(data_frame, 'relationship', 1)
    data_frame = encoding(data_frame, 'education', 1)
    data_frame = encoding(data_frame, 'income', 0)
    data_frame = encoding(data_frame, 'native-country', 1)
    data_frame = encoding(data_frame, 'workclass', 1)
    
    return data_frame

Funkcia na načítanie .csv súborov, ich spojenie podľa atribútov name a address a vyhodenie nepotrebných atribútov

In [22]:
def load_data(typ):
    if(typ == 0):
        df_other = read_csv("data/other_train.csv")
        df_personal = read_csv("data/personal_train.csv")
    elif(typ == 1):
        df_other = read_csv("data/other_valid.csv")
        df_personal = read_csv("data/personal_valid.csv")

    data_frame1 = pd.merge(df_personal, df_other, on=['name', 'address'])

    data_frame = unify_rows(data_frame1)

    data_frame = data_frame.drop('Unnamed: 0_x', 1)
    data_frame = data_frame.drop('Unnamed: 0_y', 1)

    return data_frame

Funkia detailnejšie rozďeluje atribút medical-info na ďalšie atribúty (do viacero stĺpcov), ktoré sa potom pretypujú na typ float.

In [23]:
def split_medical_info(data_frame):
    medical_line = data_frame["medical_info"].str.replace(':', ',').str.replace("'", '').str.replace("{", '').str.replace("}", '')
    medical_elements = medical_line.str.split(",", -1, expand = True)

    diabetes_dataset = {}

    for index in range(0, medical_elements.shape[1]):
        i = medical_elements[0][0]
        if(index % 2 == 0):
            diabetes_dataset[medical_elements[index][0]] = medical_elements[index + 1]

    diabetes_dataset = pd.DataFrame(diabetes_dataset)
    data_frame = pd.concat([data_frame, diabetes_dataset.reindex(data_frame.index)], axis = 1)
    data_frame = data_frame.drop(columns = ['medical_info'])
    
    data_frame['mean_glucose'] = data_frame['mean_glucose'].astype(float)
    data_frame['std_glucose'] = data_frame['std_glucose'].astype(float)
    data_frame['kurtosis_glucose'] = data_frame['kurtosis_glucose'].astype(float)
    data_frame['skewness_glucose'] = data_frame['skewness_glucose'].astype(float)
    data_frame['mean_oxygen'] = data_frame['mean_oxygen'].astype(float)
    data_frame['std_oxygen'] = data_frame['std_oxygen'].astype(float)
    data_frame['kurtosis_oxygen'] = data_frame['kurtosis_oxygen'].astype(float)
    data_frame['skewness_oxygen'] = data_frame['skewness_oxygen'].astype(float)
    
    return data_frame

Funkcia nahradzuje outliery hodnotami z 5% alebo 95% kvartilu. Použili sme Interquartile Range Method.

In [24]:
def replace_outliers(data_frame, columns):
    
    for i in range(len(columns)):
#         sns.distplot(data_frame[columns[i]])

        # calculate interquartile range
        q5, q95 = percentile(data_frame[columns[i]], 5), percentile(data_frame[columns[i]], 95)
        iqr = q95 - q5
#         print('Percentiles: 5th=%.3f, 95th=%.3f, IQR=%.3f' % (q5, q95, iqr)) 

        # calculate the outlier cutoff
        cut_off = iqr * 1.5
        lower, upper = q5 - cut_off, q95 + cut_off
#         print('cutoff=', cut_off, 'lower=', lower + cut_off, 'upper=', upper - cut_off)

        # identify outliers
        outliers = data_frame[columns[i]][(data_frame[columns[i]] > upper) | (data_frame[columns[i]] < lower)]
#         print('Identified outliers: %d' % len(outliers))

        # data without outliers
        outliers_removed = data_frame[columns[i]][(data_frame[columns[i]] <= upper) & (data_frame[columns[i]] >= lower)]
#         print('Non-outlier observations: %d' % len(outliers_removed))


        for j, value in data_frame.iterrows():
                    if value[columns[i]] < lower:
                        data_frame.loc[j, columns[i]] = lower
                    elif value[columns[i]] > upper:
                        data_frame.loc[j, columns[i]] = upper

#         sns.distplot(data_frame[columns[i]])
    
    return data_frame

Kedže niektoré dáta nemôžu byť záporné (jedná sa o štandardnú odchýlku), tak záporné hodnoty tohto atribútu zmeníme na kladné.

In [25]:
def change_to_positive(data_frame, col):
    for i in range(len(data_frame[col])):
        data = data_frame[col][i]
        if data < 0:
            data_frame.loc[i, col] = abs(data)
    return data_frame

Využitie kNN imputácie na doplnenie chýbajúcich hodnôt.

In [26]:
def knn(data_frame, columns):
    # summarize total missing
    print("Missing: " + str(data_frame.isnull().sum().sum()))

#     temp = [col for col in data_frame.columns if data_frame[col].dtypes != 'O']

    temp = columns

    # define imputer
    imputer = KNNImputer()

    # fit on the dataset
    imputer.fit(data_frame[temp])

    # transform the dataset
    data_frame[temp] = imputer.transform(data_frame[temp])
    Xtrans = imputer.transform(data_frame[temp])

    # summarize total missing
    print("Missing: " + str(sum(isnan(Xtrans).flatten())))
    
    return data_frame

Execute funkcia je hlavná funkcia, ktorá spúšťa všetky ostatné.

In [27]:
def execute():
    for i in range(2):
        df_merged = load_data(i)
        df_merged = split_medical_info(df_merged)
        df_merged = unify_values(df_merged)
        df_merged = change_date_format(df_merged,'date_of_birth')
        df_merged = change_to_positive(df_merged, 'std_glucose')

        df_merged = knn(df_merged, ['mean_glucose', 'std_glucose', 'kurtosis_glucose', 'skewness_glucose', 
                                'mean_oxygen', 'std_oxygen', 'kurtosis_oxygen', 'skewness_oxygen'])
        df_merged = fill_median(df_merged, ['education-num', 'fnlwgt', 'hours-per-week'])
        df_merged = fill_mode(df_merged, ['class', 'race', 'marital-status', 'occupation', 'pregnant', 'relationship', 
                                      'education', 'income', 'native-country', 'workclass'])
        df_merged = fill_mean(df_merged, ['capital-gain', 'capital-loss'])
        df_merged = man_not_pregnant(df_merged, 'sex', 'pregnant')
        df_merged = get_zip(df_merged,'address')
        df_merged = df_merged.drop('name', 1)
        df_merged = calculate_age(df_merged, 'date_of_birth', 'age')
        df_merged = df_merged.drop('date_of_birth', 1)
        df_merged = encode_objects(df_merged)
        df_merged = replace_outliers(df_merged, ['capital-gain', 'capital-loss', 'skewness_oxygen'])
    # # V analýze sme zistili, že v atribúte capital-loss sa všetky outliere nahradili rovnakou hodnotou, preto tento atribút 
    # # nemá žiadnu výpovednú hodnotu a je vhodné ho zmazať (zakomentovať, ak atribút mazať nechceme).
        df_merged = df_merged.drop('capital-loss', 1)
        df_merged = transform_data(df_merged, ['mean_glucose', 'std_glucose', 'kurtosis_glucose', 'skewness_glucose', 
                                'mean_oxygen', 'std_oxygen', 'kurtosis_oxygen', 'skewness_oxygen'])
    # # Uloženie upravenej dátovej sady do .csv súboru
        if i == 0:
            df_merged.to_csv(r'data\train.csv')
        elif i == 1:
            df_merged.to_csv(r'data\valid.csv')

Spustenie

In [28]:
execute()

Missing: 895
Missing: 0


c:\users\matto\appdata\local\programs\python\python38\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
c:\users\matto\appdata\local\programs\python\python38\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
c:\users\matto\appdata\local\programs\python\python38\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


Missing: 305
Missing: 0


c:\users\matto\appdata\local\programs\python\python38\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
c:\users\matto\appdata\local\programs\python\python38\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
c:\users\matto\appdata\local\programs\python\python38\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


Potrebovali sme upraviť kód tak, aby sme pre trénovací aj validačný dataset vykonali rovnaké operácie. Pôvodná funkcia execute() sa vykonávala iba raz a do jedného .csv súboru sa uložil trénovací aj validačný dataset. Teraz sme túto funkciu upravili spoločne s funkciou load_data(), do ktorej sme pridali parameter, ktorý udáva, o ktorý dataset sa jedná. Funkcia execute sa teraz vykonáva dvakrát. Najprv sa spracuje trénovací dataset, uloží sa a rovnaký proces sa vykoná aj pre validačný dataset.

Načítanie predspracovaných dát.

In [29]:
df_train = read_csv("data/train.csv")
df_valid = read_csv("data/valid.csv")

In [30]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3931 entries, 0 to 3930
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        3931 non-null   int64  
 1   address           3931 non-null   int64  
 2   age               3931 non-null   int64  
 3   sex               3931 non-null   int64  
 4   race              3931 non-null   int64  
 5   marital-status    3931 non-null   int64  
 6   occupation        3931 non-null   int64  
 7   pregnant          3931 non-null   int64  
 8   education-num     3931 non-null   float64
 9   relationship      3931 non-null   int64  
 10  capital-gain      3931 non-null   float64
 11  education         3931 non-null   int64  
 12  fnlwgt            3931 non-null   float64
 13  class             3931 non-null   float64
 14  income            3931 non-null   int64  
 15  native-country    3931 non-null   int64  
 16  hours-per-week    3931 non-null   float64


In [31]:
df_valid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1311 entries, 0 to 1310
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        1311 non-null   int64  
 1   address           1311 non-null   int64  
 2   age               1311 non-null   int64  
 3   sex               1311 non-null   int64  
 4   race              1311 non-null   int64  
 5   marital-status    1311 non-null   int64  
 6   occupation        1311 non-null   int64  
 7   pregnant          1311 non-null   int64  
 8   education-num     1311 non-null   float64
 9   relationship      1311 non-null   int64  
 10  capital-gain      1311 non-null   float64
 11  education         1311 non-null   int64  
 12  fnlwgt            1311 non-null   float64
 13  class             1311 non-null   float64
 14  income            1311 non-null   int64  
 15  native-country    1311 non-null   int64  
 16  hours-per-week    1311 non-null   float64
